# Experiment Introduction

Background, 
In order to do ablation experiment, we have diffculty like
1. most of them are same, we have to repeat many times.
2. too many model, config, code hard to manage and save. too messy


For all kind of SSL training workflow, we have to define the hyperparameters includes 4 aspect,
Dataset
Model
Training
Saving COnfig



How to Use this Experiment?



# Library

In [7]:
# Import config
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoSSL")
import yaml
from torchvision.transforms import RandomRotation,GaussianBlur,ColorJitter
from autoSSL.evaluate import eval_KNN,eval_linear,eval_KNNplot,pipe_collate
from autoSSL.models import BarlowTwins, BYOL, MoCo, SimCLR, SimSiam, VICReg ,Toymodel, pipe_model 
from autoSSL.utils import embedding_feature,ck_callback,dict2transformer,trans2multi,join_dir,ContinuousCSVLogger  
from autoSSL.data import PipeDataset
from autoSSL.train import Trainer

from lightly.transforms import SimCLRTransform
from lightly.transforms.rotation import random_rotation_transform
import torchvision.transforms as T
import pytorch_lightning as pl
pl.seed_everything(0)

Global seed set to 0


0

# Import Global Baseline Config

In [8]:
# Load the YAML file
with open('global.yaml', 'r') as file:
    global_config = yaml.safe_load(file)

# Write your experiment notebook name here
global_config["experiment"]="views VS model"   
global_config["dataset_dir"]="../Datasets/cifar10/train/"
global_config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,None,None]]    

global_config["loss_func"]="BarlowTwinsLoss"     
global_config["view_model"]="None"     
global_config["view"]=2 
global_config["stop_gradient"]=False   
global_config["optimizer"]="SGD"      
global_config["schedule"]="cos"   
global_config["model"]="Toymodel"
global_config["batch_size"]=128
global_config["input_size"]=32
global_config["max_epochs"]=50

#global_config["backbone"]="resnet18_pretrained"

# Define global Training Augmentation
global_SSL_augmentation=T.Compose([
    T.RandomResizedCrop(size=global_config["input_size"], scale=(0.08, 1.0)),
    T.RandomApply([T.RandomRotation(degrees=90)], p=0),
    T.RandomHorizontalFlip(p= 0.5),
    T.RandomVerticalFlip(p=0),
    T.RandomApply([T.ColorJitter(
            brightness=0.4,contrast=0.4,saturation=0.4, hue=0.1)], p=0.8),
    T.RandomGrayscale(p=0.2),
    T.ToTensor(),
    T.Normalize(mean= [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# Define the Testing Augmentation
test_SSL_augmentation=T.Compose([
    T.ToTensor(),
    T.Normalize(mean= [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

p_knndata= PipeDataset(input_dir=global_config["path_to_test_cifar10"], 
        augmentation=trans2multi(test_SSL_augmentation,view=1), 
        batch_size=global_config["batch_size"],num_workers=global_config["num_workers"]).dataloader
p_knndata=[p_knndata,10]  # The second number is the classes number of this datasets


## Config 1 Baseline for everything (Negative)

In [ ]:
pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
st=True


for baseline in [[pd,st,True],[[],False,False],[pd,False,True],[[],st,False]]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=2
    config["view_model"]= "None"
    config["name"]=f"LNeg_VMNone_PR{baseline[2]}_st{baseline[1]}_b128_m50_v{4}"
    config["loss_func"]="NegativeCosineSimilarity" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
    config["stop_gradient"]= baseline[1]
    config["predhead_dim"]= baseline[0]
    config["max_epochs"]= 50
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

## Config2 Grid Strategy View 4 

In [ ]:

for baseline0 in ["var_view","Stop", "mean_n" ,"fastsim", "pair-pair" , "1_n" ,"1_fastsim"]:   
    
    
    pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
    st=True
    pd2=[[2048,512, "BN","RELU"],[512,2048,"BN",None]]  
    for baseline in [[pd,st,"NoBN"]]: #,[[],False,False],[pd,False,"TruewithBN"],[[],st,False]
        # MAKE YOUR OWN CONFIG
        config=global_config.copy()
        # Fill the config
        SSL_augmentation=global_SSL_augmentation
        config["view"]=4
        config["view_model"]= baseline0
        config["name"]=f"LNeg_VM{baseline0}_PR{baseline[2]}_st{baseline[1]}_b128_m50_v{4}"
        config["loss_func"]="NegativeCosineSimilarity" 
        config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
        config["stop_gradient"]= baseline[1]
        config["predhead_dim"]= baseline[0]
        config["max_epochs"]= 50
         
        # THIS IS THE CODE TO LOAD DATASET
        pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))

        # THIS IS THE CODE TO LOAD MODEL
        #pmodel=pipe_model(config=config) 
        pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation


        # Use this if you want to START a train
        trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =1) #precision='16-mixed',
        trainer.fit(pmodel, pdata.dataloader,)  

        # Use this if you want to CONTINUE a train
        #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
        #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
        print(pmodel.debug)
        del pdata
        del pmodel
        del trainer    
        break

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:615: UserWarning: Checkpoint directory C:\Users\isxzl\OneDrive\Code\AutoSSL\experiment\experiment_checkpoints\views VS model\LNeg_VMvar_view_PRNoBN_stTrue_b128_m50_v4 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                     | Params
------------

Epoch 0:  74%|█████████████████████████████████████████████▋                | 287/390 [00:27<00:09, 10.55it/s, v_num=1]

In [ ]:
# Test
for baseline0 in ["var_view"]:   
    
    
    pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
    st=True
    pd2=[[2048,512, "BN","RELU"],[512,2048,"BN",None]]  
    for baseline in [[pd,st,"NoBN"]]: #,[[],False,False],[pd,False,"TruewithBN"],[[],st,False]
        # MAKE YOUR OWN CONFIG
        config=global_config.copy()
        # Fill the config
        SSL_augmentation=global_SSL_augmentation
        config["view"]=4
        config["view_model"]= baseline0
        config["name"]=f"Test_adam_LNeg_VM{baseline0}_PR{baseline[2]}_st{baseline[1]}_b128_m50_v{4}"
        config["loss_func"]="NegativeCosineSimilarity"
        config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
        config["stop_gradient"]= baseline[1]
        config["predhead_dim"]= baseline[0]
        config["max_epochs"]= 50
        config["optimizer"]= "Adam"
        # THIS IS THE CODE TO LOAD DATASET
        pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))

        # THIS IS THE CODE TO LOAD MODEL
        #pmodel=pipe_model(config=config) 
        pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation


        # Use this if you want to START a train
        trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =1) #precision='16-mixed',
        trainer.fit(pmodel, pdata.dataloader,)  

        # Use this if you want to CONTINUE a train
        #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
        #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
        print(pmodel.debug)
        del pdata
        del pmodel
        del trainer    
        break

## Config3: view can fix stop or not

In [ ]:
pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
pd2=[[2048,512, "BN","RELU"],[512,2048,"BN",None]]    
st=True
for baseline in [[pd2,"TrueBN"]]:  #[pd,"TruenoBN"],
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=4
    config["view_model"]= "mean_n"
    config["name"]=f"LNeg_VMmean_n_PR{baseline[1]}_stFalse_b128_m50_v{4}"
    config["loss_func"]="NegativeCosineSimilarity" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
    config["stop_gradient"]= False
    config["predhead_dim"]= baseline[0]
    config["max_epochs"]= 50
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =1) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

## Config4: Check different view

In [ ]:
pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
pd2=[[2048,512, "BN","RELU"],[512,2048,"BN",None]]    
st=True
for baseline in [6]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=baseline
    config["view_model"]= "mean_n" #"1_fastsim"  
    config["name"]=f"LNeg_VMmean_n_PRTruenoBN_stTrue_b128_m50_v{baseline}"
    config["loss_func"]="NegativeCosineSimilarity" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
    config["stop_gradient"]= True
    config["predhead_dim"]= [[2048,512, "BN","RELU"],[512,2048,None,None]]    
    config["max_epochs"]= 50

    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

## Config5: Check how it improve the baseline

In [ ]:
pd=[[2048,512, "BN","RELU"],[512,2048,None,None]]    
pd2=[[2048,512, "BN","RELU"],[512,2048,"BN",None]]    
st=True

for baseline in [["None",2]]: #"mean_n",
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=baseline[1]
    config["view_model"]= baseline[0]#"1_fastsim"  
    config["name"]=f"Model_{'VICREG'}_VM{baseline[0]}_v{baseline[1]}"
    config["loss_func"]="VICRegLoss" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
    config["stop_gradient"]= False
    #config["predhead_dim"]= [[2048,512, "BN","RELU"],[512,2048,None,None]]    
    config["max_epochs"]= 25

    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

In [9]:
 
for baseline in [["mean_n",4],["None",2]]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=baseline[1]
    config["view_model"]= baseline[0] #"1_fastsim"  
    config["name"]=f"Model_{'BT'}_VM{baseline[0]}_v{baseline[1]}"
    config["loss_func"]="BarlowTwinsLoss" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
    config["stop_gradient"]= False
    #config["predhead_dim"]= [[2048,512, "BN","RELU"],[512,2048,None,None]]    
    config["max_epochs"]= 25

    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:615: UserWarning: Checkpoint directory C:\Users\isxzl\OneDrive\Code\AutoSSL\experiment\experiment_checkpoints\views VS model\Model_BT_VMmean_n_v4 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type            | Params
------------------------------------------

Epoch 4: 100%|██████████████████████████████████████████████████████████████| 390/390 [01:59<00:00,  3.25it/s, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 9: 100%|██████████████████████████████████████████| 390/390 [02:02<00:00,  3.19it/s, v_num=0, kNN_accuracy=25.90]
Validation: 0it [00:00, ?it/s]
Epoch 14: 100%|█████████████████████████████████████████| 390/390 [02:02<00:00,  3.17it/s, v_num=0, kNN_accuracy=37.80]
Validation: 0it [00:00, ?it/s]
Epoch 19: 100%|█████████████████████████████████████████| 390/390 [02:00<00:00,  3.24it/s, v_num=0, kNN_accuracy=41.50]
Validation: 0it [00:00, ?it/s]
Epoch 24: 100%|█████████████████████████████████████████| 390/390 [02:01<00:00,  3.22it/s, v_num=0, kNN_accuracy=43.10]
Validation: 0it [00:00, ?it/s]
Epoch 24: 100%|█████████████████████████████████████████| 390/390 [02:13<00:00,  2.92it/s, v_num=0, kNN_accuracy=43.80]

`Trainer.fit` stopped: `max_epochs=25` reached.


Epoch 24: 100%|█████████████████████████████████████████| 390/390 [02:13<00:00,  2.92it/s, v_num=0, kNN_accuracy=43.80]
{'stop': 8, 'optim': [50000, 128, 25]}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:615: UserWarning: Checkpoint directory C:\Users\isxzl\OneDrive\Code\AutoSSL\experiment\experiment_checkpoints\views VS model\Model_BT_VMNone_v2 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type            | Params
--------------------------------------------

Epoch 4: 100%|██████████████████████████████████████████████████████████████| 390/390 [00:48<00:00,  8.02it/s, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 9: 100%|██████████████████████████████████████████| 390/390 [00:49<00:00,  7.88it/s, v_num=0, kNN_accuracy=26.30]
Validation: 0it [00:00, ?it/s]
Epoch 14: 100%|█████████████████████████████████████████| 390/390 [00:47<00:00,  8.14it/s, v_num=0, kNN_accuracy=40.60]
Validation: 0it [00:00, ?it/s]
Epoch 19: 100%|█████████████████████████████████████████| 390/390 [00:48<00:00,  7.97it/s, v_num=0, kNN_accuracy=45.50]
Validation: 0it [00:00, ?it/s]
Epoch 24: 100%|█████████████████████████████████████████| 390/390 [00:49<00:00,  7.84it/s, v_num=0, kNN_accuracy=47.80]
Validation: 0it [00:00, ?it/s]
Epoch 24: 100%|█████████████████████████████████████████| 390/390 [01:01<00:00,  6.32it/s, v_num=0, kNN_accuracy=49.00]

`Trainer.fit` stopped: `max_epochs=25` reached.


Epoch 24: 100%|█████████████████████████████████████████| 390/390 [01:01<00:00,  6.32it/s, v_num=0, kNN_accuracy=49.00]
{'stop': 2, 'optim': [50000, 128, 25]}


In [4]:
for baseline in [["mean_n",4],["None",2]]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["view"]=baseline[1]
    config["view_model"]=baseline[0] #"1_fastsim"  
    config["name"]=f"Model_{'SimCLR'}_VM{baseline[0]}_v{baseline[1]}"
    config["loss_func"]="SimCLR" 
    config["prjhead_dim"]=[[512,2048, "BN","RELU"],[2048,2048,"BN",None]]    
    config["stop_gradient"]= False
    #config["predhead_dim"]= [[2048,512, "BN","RELU"],[512,2048,None,None]]    
    config["max_epochs"]= 25

    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=trans2multi(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:615: UserWarning: Checkpoint directory C:\Users\isxzl\OneDrive\Code\AutoSSL\experiment\experiment_checkpoints\views VS model\Model_SimCLR_VMmean_n_v4 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type           | Params
---------------------------------------

Epoch 4: 100%|██████████████████████████████████████████████████████████████| 390/390 [00:36<00:00, 10.62it/s, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 9: 100%|██████████████████████████████████████████| 390/390 [00:38<00:00, 10.12it/s, v_num=0, kNN_accuracy=54.30]
Validation: 0it [00:00, ?it/s]
Epoch 14: 100%|█████████████████████████████████████████| 390/390 [00:37<00:00, 10.34it/s, v_num=0, kNN_accuracy=58.70]
Validation: 0it [00:00, ?it/s]
Epoch 19: 100%|█████████████████████████████████████████| 390/390 [00:36<00:00, 10.83it/s, v_num=0, kNN_accuracy=62.80]
Validation: 0it [00:00, ?it/s]
Epoch 24: 100%|█████████████████████████████████████████| 390/390 [00:39<00:00,  9.83it/s, v_num=0, kNN_accuracy=65.30]
Validation: 0it [00:00, ?it/s]
Epoch 24: 100%|█████████████████████████████████████████| 390/390 [00:52<00:00,  7.42it/s, v_num=0, kNN_accuracy=66.50]

`Trainer.fit` stopped: `max_epochs=25` reached.


Epoch 24: 100%|█████████████████████████████████████████| 390/390 [00:52<00:00,  7.42it/s, v_num=0, kNN_accuracy=66.50]
{'stop': 8, 'optim': [50000, 128, 25]}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
C:\Users\isxzl\anaconda3\envs\AutoGPT\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:615: UserWarning: Checkpoint directory C:\Users\isxzl\OneDrive\Code\AutoSSL\experiment\experiment_checkpoints\views VS model\Model_SimCLR_VMNone_v2 exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type           | Params
-----------------------------------------

Epoch 4: 100%|██████████████████████████████████████████████████████████████| 390/390 [00:25<00:00, 15.01it/s, v_num=0]
Validation: 0it [00:00, ?it/s]
Epoch 9: 100%|██████████████████████████████████████████| 390/390 [00:24<00:00, 16.00it/s, v_num=0, kNN_accuracy=45.30]
Validation: 0it [00:00, ?it/s]
Epoch 14: 100%|█████████████████████████████████████████| 390/390 [00:24<00:00, 15.89it/s, v_num=0, kNN_accuracy=49.20]
Validation: 0it [00:00, ?it/s]
Epoch 19: 100%|█████████████████████████████████████████| 390/390 [00:23<00:00, 16.87it/s, v_num=0, kNN_accuracy=52.30]
Validation: 0it [00:00, ?it/s]
Epoch 24: 100%|█████████████████████████████████████████| 390/390 [00:24<00:00, 15.87it/s, v_num=0, kNN_accuracy=54.90]
Validation: 0it [00:00, ?it/s]
Epoch 24: 100%|█████████████████████████████████████████| 390/390 [00:36<00:00, 10.56it/s, v_num=0, kNN_accuracy=55.90]

`Trainer.fit` stopped: `max_epochs=25` reached.


Epoch 24: 100%|█████████████████████████████████████████| 390/390 [00:36<00:00, 10.56it/s, v_num=0, kNN_accuracy=55.90]
{'stop': 2, 'optim': [50000, 128, 25]}


## Improved Trainer

In [ ]:

for baseline in [4,12]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["name"]=f"mean_n_BT_Baseline_b128_m50_v{str(baseline)}"

    config["view"]=baseline
    config["view_model"]= "mean_n"   # 1_n # mean_n #1_fastsim
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=dict2transformer(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    


## Config 5 

In [ ]:
 
for baseline in [4,12]:
    # MAKE YOUR OWN CONFIG
    config=global_config.copy()
    # Fill the config
    SSL_augmentation=global_SSL_augmentation
    config["name"]=f"1_fastsim_BT_Baseline_b128_m50_v{str(baseline)}"

    config["view"]=baseline
    config["view_model"]= "1_fastsim"   # 1_n # mean_n #
    # THIS IS THE CODE TO LOAD DATASET
    pdata= PipeDataset(config=config,augmentation=dict2transformer(SSL_augmentation,view=config["view"]))
    
    # THIS IS THE CODE TO LOAD MODEL
    #pmodel=pipe_model(config=config) 
    pmodel=pipe_model(config=config,MonitoringbyKNN=p_knndata) # All save the validation
    
 
    # Use this if you want to START a train
    trainer=Trainer(config, model_mode="start",check_val_every_n_epoch =5) #precision='16-mixed',
    trainer.fit(pmodel, pdata.dataloader,)  
    
    # Use this if you want to CONTINUE a train
    #trainer=Trainer(config, model_mode="continue", extra_epoch=1,precision='16-mixed')
    #trainer.fit(pmodel, pdata.dataloader,ckpt_path="latest")  
    print(pmodel.debug)
    del pdata
    del pmodel
    del trainer    


# Evaluation

In [10]:
 
from sklearn.preprocessing import normalize
import os
import glob
import re
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
from autoSSL.models.Backbone import pipe_backbone
from sklearn.metrics import confusion_matrix
import seaborn as sns

def embed(x, embedding_model, device):
    embedding_model.eval()
    embedding_model.to(device)
    x = x.float().to(device)

    with torch.no_grad():
        embeddings = embedding_model(x)
        pooled_embeddings = torch.nn.functional.adaptive_avg_pool2d(embeddings, (1, 1))

    return pooled_embeddings.view(pooled_embeddings.size(0), -1).cpu().numpy()

def eval_everything(pipe_data, models, device='cuda', split=None, test=None, baseline=None):
    if split is not None:
        train_data, test_data = pipe_data.split(split)
    elif test is None:
        train_data = pipe_data
        test_data = pipe_data
    else:         
        train_data = pipe_data
        test_data = test

    print("Load the training and testing dataset")
    X_train, y_train = train_data.array[0], train_data.array[1]
    X_test, y_test = test_data.array[0], test_data.array[1]

    if isinstance(models, torch.nn.Module):
        models = {'name': ['model_0'], 'model': [models], 'address': None} 
    elif isinstance(models, list):
        models = {'name': ['model_'+str(i) for i in range(len(models))], 'model': models, 'address': None}
     
    baselines=[]
    results = []
    baselines_name=[]
    if baseline:
        for base in baseline:
            baseline_backbone, _ = pipe_backbone(backbone=base)
            models['model'].append(baseline_backbone)
            models['name'].append('baseline_' + base)
            baselines_name.append('baseline_' + base)
            baselines.append(baseline_backbone)
        
    writer = pd.ExcelWriter(models['address'].replace('.csv', '_confusion.xlsx'))

    for i, embedding_model in enumerate(tqdm(models['model'])):
        if embedding_model in baselines:
            pass
        else:
            embedding_model=embedding_model.backbone
        X_train_embedding = [embed(x, embedding_model, device) for x in DataLoader(X_train, batch_size=16)]
        X_train_embedding = np.concatenate(X_train_embedding)

        X_test_embedding = [embed(x, embedding_model, device) for x in DataLoader(X_test, batch_size=16)]
        X_test_embedding = np.concatenate(X_test_embedding)

        X_train_embedding = normalize(X_train_embedding)  # Normalize train embeddings

        X_test_embedding = normalize(X_test_embedding)  # Normalize test embeddings

        
        if X_test_embedding is None:
            accuracy = 'model_collapse'
            confusion = None
        else:
            clf = SGDClassifier(loss='log_loss')

            clf.fit(X_train_embedding, y_train)

            # Get class probabilities for each sample
            class_probs = clf.predict_proba(X_test_embedding)

            # Get the top 1 predictions
            top1_preds = np.argmax(class_probs, axis=1)
            top3_preds = np.argpartition(class_probs, -3, axis=1)[:,-3:]

            # Calculate confusion matrix
            confusion = confusion_matrix(y_test, top1_preds)
            # Normalize confusion matrix by row (i.e by the number of samples in each class)
            confusion = confusion.astype('float') / confusion.sum(axis=1)[:, np.newaxis]
            # Average accuracy is the mean of the diagonal elements (the correctly classified instances)
            top1_average_accuracy = np.mean(np.diag(confusion))
            
            # Calculate accuracy
            top1_accuracy = accuracy_score(y_test, top1_preds)
            top3_accuracy = np.mean([1 if y in top3 else 0 for y, top3 in zip(y_test, top3_preds)])
                            
            # K-Nearest Neighbors classifier
            knn = KNeighborsClassifier(n_neighbors=20)
            knn.fit(X_train_embedding, y_train)
            knn_preds = knn.predict(X_test_embedding)
            knn_accuracy = accuracy_score(y_test, knn_preds)

            accuracy = {
                "Top-1 Accuracy": top1_accuracy,
                "Top-3 Accuracy": top3_accuracy,
                "Top-1 Average Accuracy": top1_average_accuracy,
                "KNN Top-1 Accuracy": knn_accuracy  # KNN accuracy
            }

        namee = models["name"][i]
        results.append((namee, accuracy))

        
        del embedding_model
        torch.cuda.empty_cache()
        
        # Save confusion matrix to Excel file
        if confusion is not None:
            df_confusion = pd.DataFrame(confusion)
            df_confusion.to_excel(writer, sheet_name=namee)

    writer.save()

    if models['address'] is not None:
        df = pd.read_csv(models['address'])

        # If baselines are present, add new rows in the dataframe for them
        if baselines:
            for base, base_name in zip(baselines, baselines_name):
                # Initialize a new row with default values
                new_row = {col: None for col in df.columns}
                # Update the values we know
                new_row.update({
                    'dir_name': base_name,
                })
                # Append the new row to the dataframe
                df = df.append(new_row, ignore_index=True)

        # Assuming the results are in the same order as the models in the dataframe
        df['linear_top1_accuracy'] = [result[1]["Top-1 Accuracy"] for result in results]
        df['linear_top3_accuracy'] = [result[1]["Top-3 Accuracy"] for result in results]
        df['linear_top1_average_accuracy'] = [result[1]["Top-1 Average Accuracy"] for result in results]
        df['linear_knn_top1_accuracy'] = [result[1]["KNN Top-1 Accuracy"] for result in results]

        df.to_csv(models['address'], index=False)

    return results


In [11]:
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoSSL")
from autoSSL.data import PipeDataset 
from autoSSL.utils import  dict2transformer,trans2multi  
import torchvision.transforms as T
from autoSSL.evaluate import eval_KNN,eval_linear,eval_KNNplot,pipe_collate#,eval_everything
import yaml
# Load the YAML file
with open('global.yaml', 'r') as file:
    global_config = yaml.safe_load(file)


test_augmentation=T.Compose([
    T.ToTensor(),
    T.Normalize(mean= [0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# "LNeg_VM.*_PRTruewithBN_stTrue_b128_m50_v4"

collate =pipe_collate(address="experiment_checkpoints/views VS model/", reg="Model_*")

pdata = PipeDataset(input_dir=global_config["path_to_test_cifar10"],
    augmentation=trans2multi(test_augmentation,view=1))

 

Collating the models' (evaluating) information to experiment_checkpoints/views VS model/Model__.csv


In [12]:
aaa=eval_everything(pdata, models=collate, device=global_config["device"])


Load the training and testing dataset


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:37<00:00,  4.66s/it]
C:\Users\isxzl\AppData\Local\Temp\ipykernel_27348\357591954.py:127: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [13]:
aaa

[('Model_BT_VMmean_n_v4',
  {'Top-1 Accuracy': 0.42515748425157485,
   'Top-3 Accuracy': 0.7487251274872513,
   'Top-1 Average Accuracy': 0.425151948051948,
   'KNN Top-1 Accuracy': 0.4520547945205479}),
 ('Model_BT_VMNone_v2',
  {'Top-1 Accuracy': 0.4844515548445156,
   'Top-3 Accuracy': 0.8134186581341866,
   'Top-1 Average Accuracy': 0.48445434565434564,
   'KNN Top-1 Accuracy': 0.50994900509949}),
 ('Model_SimCLR_VMmean_n_v4',
  {'Top-1 Accuracy': 0.6398360163983602,
   'Top-3 Accuracy': 0.9048095190480951,
   'Top-1 Average Accuracy': 0.6398301698301698,
   'KNN Top-1 Accuracy': 0.650934906509349}),
 ('Model_SimCLR_VMNone_v2',
  {'Top-1 Accuracy': 0.5495450454954505,
   'Top-3 Accuracy': 0.8661133886611339,
   'Top-1 Average Accuracy': 0.5495375624375625,
   'KNN Top-1 Accuracy': 0.5729427057294271}),
 ('Model_Simsiam_VMmean_n_v4',
  {'Top-1 Accuracy': 0.6964303569643036,
   'Top-3 Accuracy': 0.9244075592440756,
   'Top-1 Average Accuracy': 0.6964298701298702,
   'KNN Top-1 Accura